# 第三章第二节：生成器：容器


## 本节的目的

生成器经常需要处理可变数量的对象，无论它们是IO，模块还是测试向量。
容器（collections）是处理此类情况的重要基础。
本模块将介绍Scala容器以及如何将它们与Chisel生成器一起使用。


## 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))


注意，下面我们多加了一个新的导入，因为`mutable.ArrayBuffer`在`scala.collections`之中。

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import scala.collection._

---
# 生成器和容器<a name="generators-and-collections"></a> 

在本节中，我们将重点介绍*生成器（generators）*的概念以及怎样用Scala容器来作为实现它们的工具。
与其将Chisel代码看作是一个电路的*实例（instance）*（即特定电路的描述），
我们在这里将其视为电路的生成器。

我们将首先来看看先前练习中的FIR滤波器。

In [3]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + x_n3 * b3.U(8.W)
}


defined class My4ElementFir

该电路是生成器的一种简单的例子，因为它可以使用不同的参数来生成此4抽头（tap）滤波器的版本：
但是，如果我们希望电路具有更多的抽头呢？我们将分几个步骤来进行：

- 建立抽头（tap）数目可配置的FIR的软件的*黄金模型（Golden Model）* 。
- 重新设计我们的测试，并确认这个新的模型它可以工作。
- 重构我们的My4ElementFir以允许可配置的抽头数量。
- 使用我们的新测试约束来测试新电路。

<span style="color:blue"> **示例：FIR黄金模型（Golden Model）** </span><br><a name="fir-golden-model"></a> 
以下是用Scala软件实现的FIR电路。

In [4]:
/**
  * 带有任意抽头（tap）的FIR滤波器的简单实现。
  */
class ScalaFirFilter(taps: Seq[Int]) {
  var pseudoRegisters = List.fill(taps.length)(0)

  def poke(value: Int): Int = {
    pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)
    var accumulator = 0
    for(i <- taps.indices) {
      accumulator += taps(i) * pseudoRegisters(i)
    }
    accumulator
  }
}

defined class ScalaFirFilter

### Seq
注意，`taps`已成为`Seq[Int]`，这意味着用户在构造该类时可以传递任意长的`Int`序列。

### 寄存器
通过`  var pseudoRegisters = List.fill(taps.length)(0)`，我们创建了一个列表`List`，它将保存前一个时钟周期的值。之所以选择列表`List`，是因为往其头部添加元素以及从尾部删除最后一个元素的语法非常简单。几乎任何scala容器中的类都可以使用。另外，我们还将此列表初始化为全零。

### Poke
我们的类添加了一个`poke`函数/方法，该函数模拟了将新输入放入滤波器并行进一个时钟周期。

### 更新寄存器
`pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)`这一行首先使用列表的`take`方法取出列表中除最后一个元素之外的所有元素，然后使用`::`这个列表的连接运算符将`value`添加到列表的开头。

### 计算输出
一个带有累加器的简单for循环将列表中的每个元素乘以其相应的抽头（tap）系数。仅带有`accumulator`的这一行将返回该值作为函数结果。


## 修改我们之前的测试用于这个黄金模型
现在，我们将使用以前的工作来确认我们的黄金模型有效。进行一些编辑就可以将我们之前的测试方法变形为...

In [5]:
val filter = new ScalaFirFilter(Seq(1, 1, 1, 1))

var out = 0

out = filter.poke(1)
println(s"out = $out")
assert(out == 1)  // 1, 0, 0, 0

out = filter.poke(4)
assert(out == 5)  // 4, 1, 0, 0
println(s"out = $out")

out = filter.poke(3)
assert(out == 8)  // 3, 4, 1, 0
println(s"out = $out")

out = filter.poke(2)
assert(out == 10)  // 2, 3, 4, 1
println(s"out = $out")

out = filter.poke(7)
assert(out == 16)  // 7, 2, 3, 4
println(s"out = $out")

out = filter.poke(0)
assert(out == 12)  // 0, 7, 2, 3
println(s"out = $out")

out = 1
out = 5
out = 8
out = 10
out = 16
out = 12


filter: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@44efb28a
out: Int = 12


执行前面的代码表明，我们的软件模型返回的结果与之前的`My4ElementFir`相同。


## 使用黄金模型（golden model）来测试电路。<a name="use-golden-model-as-test"></a> 
既然我们对黄金模型有足够的信心，我们就可以重写测试从而将电路输出与黄金模型的输出进行比较，而不必使用费力的手工生成的测试用例。
接下来是快速实现的第一步。

In [6]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new My4ElementFir(1, 1, 1, 1)) {
  c => new PeekPokeTester(c) {
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

[info] [0.004] Elaborating design...
[info] [0.969] Done elaborating.
Total FIRRTL Compile Time: 598.5 ms
Total FIRRTL Compile Time: 80.6 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1572488622080
test cmd2HelperMy4ElementFir Success: 100 tests passed in 105 cycles taking 0.142455 seconds
[info] [0.118] RAN 100 CYCLES PASSED


goldenModel: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@b7d7ea1
res5_1: Boolean = true


上面的测试运行100个时钟周期，并在每一步检查用两种不同的方法（硬件和软件）得到的结果是否一致。

### 注意事项
（实际上是我们在撰写本文时实际犯的错误。）

1. 将`step`放在正确的位置。软件和硬件的执行方式不同；这很容易弄错。
1. 此测试很弱，因为它对IO和寄存器的大小非常敏感。用软件来实现任意数据宽度的环绕（wrap）行为非常复杂。在这里，我们确保只传递适合的值。

<span style="color:blue">**示例：参数化FIR生成器**</span><br><a name="fir-golden-model"></a> 
在下面，我们创建了一个新的滤波器的类，即`MyManyElementsFilter`，该类接受一个常量`Seq`用于抽头。该列表可以包含任意数量的元素。
另外，我们添加了`bitWidth`，用于控制电路可以处理的数字的大小。

为了使得长度可变，我们不得不重构寄存器的创建及其连接方式。
下面只用了所有容器的方法中的一部分。
后面的几节将说明如何更简洁同时也更清晰地表达该行为。

In [7]:
class MyManyElementFir(consts: Seq[Int], bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      muls += regs(i) * consts(i).U
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan.last
}

defined class MyManyElementFir

#### 我们是怎么做到的
从第7、13和18行开始，共有三个并行的部分。
我们使用一种名为`ArrayBuffer`的Scala容器类型。
`ArrayBuffer`允许您使用`+=`运算符来添加元素（也可以插入和删除，但我们不需要）。
首先，我们创建一个`ArrayBuffer`的`regs`，其元素将为`UInt`类型。
然后遍历抽头，将输入添加为第一个元素，然后使用`RegNext`创建寄存器，将该寄存器的输入连接到前一个寄存器（`regs(i-1)`），并将其初始化为无符号的0（`0.U`）。
这些寄存器用来保留之前输入的值。

接下来，我们创建另一个`UInt`类型的另一个`ArrayBuffer`-`muls`。
`muls`的每个元素都是一个节点，其第i个元素是`regs(i)`和 `const(i)`的乘积。

注意`scan.last`方法的使用。
它用来获取容器的最后一个元素，是之前构造`regs`时采用的`regs(i - 1)`的另一种更优雅的替代方案。

### 它的行为与`My4ElementFir`相同吗？
新版本的第一个测试是看它是否可以通过我们刚刚应于`My4ElementFir`的测试。
我们创建一个`MyManyElementFir`的实例，并通过它运行更多数据。

In [8]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new MyManyElementFir(Seq(1, 1, 1, 1), 8)) {
  c => new PeekPokeTester(c) {
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

[info] [0.000] Elaborating design...
[info] [0.014] Done elaborating.
Total FIRRTL Compile Time: 58.3 ms
Total FIRRTL Compile Time: 55.5 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572488626575
test cmd6HelperMyManyElementFir Success: 100 tests passed in 105 cycles taking 0.061406 seconds
[info] [0.059] RAN 100 CYCLES PASSED


goldenModel: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@2848fd38
res7_1: Boolean = true

### 现在让我们测试一堆不同大小的FIR滤波器
我们创建了一些辅助函数：`r`用来生成一个随机数；`runOneTest`会为一组特定的抽头（tap）创建一个黄金软件模型和一个滤波器的硬件仿真，然后运行至少两倍于抽头的数目的测试数据。

In [9]:
/** 用于生成随机数的方便函数 
  */
def r(): Int = {
  scala.util.Random.nextInt(1024)
}

/**
  * 运行测试对比软件和硬件的滤波器输出 
  * 运行次数至少抽头数目的两倍 2 * taps.length
  */
def runOneTest(taps: Seq[Int]) {
  val goldenModel = new ScalaFirFilter(taps)

  Driver(() => new MyManyElementFir(taps, 32)) {
    c => new PeekPokeTester(c) {
      for(i <- 0 until 2 * taps.length) {
        val input = r()

        val goldenModelResult = goldenModel.poke(input)

        poke(c.io.in, input)

        expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

        step(1)
      }
    }
  }
}

for(tapSize <- 2 until 100 by 10) {
  val taps = Seq.fill(tapSize)(r())  // 创建一系列的随机参数

  runOneTest(taps)
}

[info] [0.000] Elaborating design...
[info] [0.074] Done elaborating.
Total FIRRTL Compile Time: 46.8 ms
Total FIRRTL Compile Time: 39.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572488627334
test cmd6HelperMyManyElementFir Success: 4 tests passed in 9 cycles taking 0.006255 seconds
[info] [0.005] RAN 4 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.012] Done elaborating.
Total FIRRTL Compile Time: 92.4 ms
Total FIRRTL Compile Time: 71.0 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572488627591
test cmd6HelperMyManyElementFir Success: 24 tests passed in 29 cycles taking 0.041233 seconds
[info] [0.036] RAN 24 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.015] Done elaborating.
Total FIRRTL Compile Time: 108.6 ms
Total FIRRTL Compile Time: 111.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572488627969
test cmd6HelperMyManyElementFir Success: 44 tests passed in 49 cycles taking 0.07

defined function r
defined function runOneTest

### 只是为了好玩，我们来做一个更大的FIR滤波器
以下将在500抽头上运行单个测试
FIR滤波器。运行可能需要一分钟左右的时间。

In [10]:
runOneTest(Seq.fill(500)(r()))

[info] [0.000] Elaborating design...
[info] [0.095] Done elaborating.
Total FIRRTL Compile Time: 708.0 ms
Total FIRRTL Compile Time: 630.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572488633741
test cmd6HelperMyManyElementFir Success: 1000 tests passed in 1005 cycles taking 8.624099 seconds
[info] [8.571] RAN 1000 CYCLES PASSED


In [11]:
val taps = Seq.fill(500)(r())

val goldenModel = new ScalaFirFilter(taps)

Driver(() => new MyManyElementFir(taps, 32)) {
  c => new PeekPokeTester(c) {
    for(i <- 0 until 100) {
      val input = r()

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

[info] [0.000] Elaborating design...
[info] [0.068] Done elaborating.
Total FIRRTL Compile Time: 534.3 ms
Total FIRRTL Compile Time: 564.5 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572488645095
test cmd6HelperMyManyElementFir Success: 100 tests passed in 105 cycles taking 0.972853 seconds
[info] [0.935] RAN 100 CYCLES PASSED


taps: Seq[Int] = List(
  848,
  161,
  217,
  157,
  669,
  383,
  47,
  752,
  294,
  104,
  604,
  842,
  188,
  221,
  292,
  724,
  995,
  250,
  52,
  923,
  923,
  514,
  12,
  641,
  622,
  660,
  34,
  903,
  631,
  842,
  1012,
  83,
  236,
  71,
  197,
  202,
  962,
  816,
...
goldenModel: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@46f36148
res10_2: Boolean = true

---
# 硬件容器

<span style="color:blue"> **示例：在我们的FIR滤波器中添加运行时可配置的抽头（tap）** </span><br>
下面的代码在FIR生成器的IO里添加了一个额外的`consts`向量，该向量允许在电路生成后从外部更改参数。
这是通过Chisel的容器类型`Vec`完成的。
`Vec`支持许多Scala容器的方法，但它只能包含Chisel硬件元素。
`Vec`仅应该在普通Scala容器用不了的情况下使用。
基本上，这是以下两种情况之一：
1. 您需要在Bundle（通常是将用作IO的Bundle）中使用容器。
1. 您需要通过属于硬件的索引来访问容器（例如寄存器组register file）。

In [12]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = RegInit(VecInit(Seq.fill(length - 1)(0.U(8.W))))
  for(i <- 0 until length - 1) {
      if(i == 0) regs(i) := io.in
      else       regs(i) := regs(i - 1)
  }
  
  val muls = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) muls(i) := io.in * io.consts(i)
      else       muls(i) := regs(i - 1) * io.consts(i)
  }

  val scan = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) scan(i) := muls(i)
      else scan(i) := muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

defined class MyManyDynamicElementVecFir

In [13]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new MyManyDynamicElementVecFir(4)) {
  c => new PeekPokeTester(c) {
    poke(c.io.consts(0), 1)
    poke(c.io.consts(1), 1)
    poke(c.io.consts(2), 1)
    poke(c.io.consts(3), 1)
    for(i <- 0 until 100) {
      val input = scala.util.Random.nextInt(8)

      val goldenModelResult = goldenModel.poke(input)

      poke(c.io.in, input)

      expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

      step(1)
    }
  }
}

[info] [0.000] Elaborating design...
[info] [0.070] Done elaborating.
Total FIRRTL Compile Time: 66.3 ms
Total FIRRTL Compile Time: 42.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572488648836
test cmd11HelperMyManyDynamicElementVecFir Success: 100 tests passed in 105 cycles taking 0.038679 seconds
[info] [0.037] RAN 100 CYCLES PASSED


goldenModel: ScalaFirFilter = ammonite.$sess.cmd3$Helper$ScalaFirFilter@5a7fc4e0
res12_1: Boolean = true

<span style="color:red">**练习：32位RISC-V处理器**</span><br><a name="fir-golden-model"></a>


[寄存器组（register file）](https://en.wikipedia.org/wiki/Register_file)是处理器中重要的组成部分。
寄存器组是可以通过多个读或写端口读取或写入的寄存器阵列。
每个端口都包含一个地址和数据字段。

[RISC-V指令集体系结构](https://riscv.org/specifications/)定义了多个指令集的变体，其中最简单的变体称为RV32I。
RV32I包含一个具有32个32位寄存器的寄存器阵列。
**无论您在索引0的位置写了什么，索引0处的寄存器（第一个寄存器）读取值始终为零**（这是为了方便的使用0这个值）。

这个练习为RV32I实现寄存器组（register file），它具有一个写入端口和若干个读取端口（读取端口的数目是参数）。
仅在`wen`（写使能）被置上时才执行写操作。

In [ ]:
class RegisterFile(readPorts: Int) extends Module {
    require(readPorts >= 0)
    val io = IO(new Bundle {
        val wen   = Input(Bool())
        val waddr = Input(UInt(5.W))
        val wdata = Input(UInt(32.W))
        val raddr = Input(Vec(readPorts, UInt(5.W)))
        val rdata = Output(Vec(readPorts, UInt(32.W)))
    })
    
    // 包含UInt类型向量的寄存器
    val reg = RegInit(VecInit(Seq.fill(32)(0.U(32.W))))

}

In [ ]:
chisel3.iotesters.Driver(() => new RegisterFile(2) ) { c => new PeekPokeTester(c) {
    def readExpect(addr: Int, value: Int, port: Int = 0): Unit = {
        poke(c.io.raddr(port), addr)
        expect(c.io.rdata(port), value)
    }
    def write(addr: Int, value: Int): Unit = {
        poke(c.io.wen, 1)
        poke(c.io.wdata, value)
        poke(c.io.waddr, addr)
        step(1)
        poke(c.io.wen, 0)
    }
    // 初始化的时候所有东西都应该是 0
    for (i <- 0 until 32) {
        readExpect(i, 0, port = 0)
        readExpect(i, 0, port = 1)
    }

    // 写入 5 * addr + 3
    for (i <- 0 until 32) {
        write(i, 5 * i + 3)
    }

    // 检查上面的写入是否成功
    for (i <- 0 until 32) {
        readExpect(i, if (i == 0) 0 else 5 * i + 3, port = i % 2)
    }

}}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
    when (io.wen) {
        reg(io.waddr) := io.wdata
    }
    for (i &lt;- 0 until readPorts) {
        when (io.raddr(i) === 0.U) {
            io.rdata(i) := 0.U
        } .otherwise {
            io.rdata(i) := reg(io.raddr(i))
        }
    }

</pre></article></div></section></div>

---
# 本节结束!

[返回顶部](#top)